<a href="https://colab.research.google.com/github/thak123/winter-school-ffzg-2023/blob/main/1_Day_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 1. Lets say hello

In [ ]:
print("hello world")

hello world


In [ ]:
!python -V

Python 3.8.16


## 2. Requirements

### Installing packages

In [ ]:
pip install -U scikit-learn pandas numpy matplotlib

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.2/12.2 MB 73.9 MB/s eta 0:00:00
  Attempting uninstall: pandas
    Found existing installation: pandas 1.3.5
    Uninstalling pandas-1.3.5:
      Successfully uninstalled pandas-1.3.5


## 3. Importing the libraries

In [20]:
import numpy as np
import re
import nltk
from sklearn.datasets import load_files
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')
import pickle
from nltk.corpus import stopwords
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


## 3.a. Code to download data file

In [3]:
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import TfidfVectorizer
from time import time

categories = [
    "alt.atheism",
    "talk.religion.misc",
    "comp.graphics",
    "sci.space",
]


In [26]:
# newsgroup_data = fetch_20newsgroups(
#         subset="all",
#         categories=categories,
#         shuffle=True,
#         random_state=42,
#     )

data_train = fetch_20newsgroups(
        subset="train",
        categories=categories,
        shuffle=True,
        random_state=42,
        # remove=remove,
    )

data_test = fetch_20newsgroups(
        subset="test",
        categories=categories,
        shuffle=True,
        random_state=42,
        # remove=remove,
    )
  

In [42]:
# order of labels in `target_names` can be different from `categories`
target_names = data_train.target_names

# split target in a training set and a test set
y_train, y_test = data_train.target, data_test.target



In [8]:

# def size_mb(docs):
#     return sum(len(s.encode("utf-8")) for s in docs) / 1e6


# def load_dataset(verbose=False, remove=()):
#     """Load and vectorize the 20 newsgroups dataset."""

#     data_train = fetch_20newsgroups(
#         subset="train",
#         categories=categories,
#         shuffle=True,
#         random_state=42,
#         remove=remove,
#     )

#     data_test = fetch_20newsgroups(
#         subset="test",
#         categories=categories,
#         shuffle=True,
#         random_state=42,
#         remove=remove,
#     )

#     # order of labels in `target_names` can be different from `categories`
#     target_names = data_train.target_names

#     # split target in a training set and a test set
#     y_train, y_test = data_train.target, data_test.target

#     # Extracting features from the training data using a sparse vectorizer
#     t0 = time()
#     vectorizer = TfidfVectorizer(
#         sublinear_tf=True, max_df=0.5, min_df=5, stop_words="english"
#     )
#     X_train = vectorizer.fit_transform(data_train.data)
#     duration_train = time() - t0

#     # Extracting features from the test data using the same vectorizer
#     t0 = time()
#     X_test = vectorizer.transform(data_test.data)
#     duration_test = time() - t0

#     feature_names = vectorizer.get_feature_names_out()

#     if verbose:

#         # compute size of loaded data
#         data_train_size_mb = size_mb(data_train.data)
#         data_test_size_mb = size_mb(data_test.data)

#         print(
#             f"{len(data_train.data)} documents - "
#             f"{data_train_size_mb:.2f}MB (training set)"
#         )
#         print(f"{len(data_test.data)} documents - {data_test_size_mb:.2f}MB (test set)")
#         print(f"{len(target_names)} categories")
#         print(
#             f"vectorize training done in {duration_train:.3f}s "
#             f"at {data_train_size_mb / duration_train:.3f}MB/s"
#         )
#         print(f"n_samples: {X_train.shape[0]}, n_features: {X_train.shape[1]}")
#         print(
#             f"vectorize testing done in {duration_test:.3f}s "
#             f"at {data_test_size_mb / duration_test:.3f}MB/s"
#         )
#         print(f"n_samples: {X_test.shape[0]}, n_features: {X_test.shape[1]}")

#     return X_train, X_test, y_train, y_test, feature_names, target_names

In [29]:
from nltk.stem import WordNetLemmatizer

stemmer = WordNetLemmatizer()
def preprocess(X):
    documents = []
    for sen in range(0, len(X)):
        # Remove all the special characters
        document = re.sub(r'\W', ' ', str(X[sen]))
        
        # remove all single characters
        document = re.sub(r'\s+[a-zA-Z]\s+', ' ', document)
        
        # Remove single characters from the start
        document = re.sub(r'\^[a-zA-Z]\s+', ' ', document) 
        
        # Substituting multiple spaces with single space
        document = re.sub(r'\s+', ' ', document, flags=re.I)
        
        # Removing prefixed 'b'
        document = re.sub(r'^b\s+', '', document)
        
        # Converting to Lowercase
        document = document.lower()
        
        # Lemmatization
        document = document.split()

        document = [stemmer.lemmatize(word) for word in document]
        document = ' '.join(document)
        
        documents.append(document)
    return documents

In [30]:
train_documents = preprocess(data_train.data)
test_documents = preprocess(data_test.data)

In [31]:
len(train_documents),len(test_documents)

(2034, 1353)

In [ ]:
# Extracting features from the training data using a sparse vectorizer
# vectorizer = TfidfVectorizer(
#     sublinear_tf=True, max_df=0.5, min_df=5, stop_words="english"
# )
# X_train = vectorizer.fit_transform(data_train.data)

# # Extracting features from the test data using the same vectorizer
# X_test = vectorizer.transform(data_test.data)

# feature_names = vectorizer.get_feature_names_out()

In [ ]:
train_documents

In [38]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(max_features=1500, min_df=5, max_df=0.7, stop_words=stopwords.words('english'))
X_train = vectorizer.fit_transform(train_documents).toarray()

In [39]:
feature_names = vectorizer.get_feature_names_out()

In [40]:
from sklearn.feature_extraction.text import TfidfTransformer
tfidfconverter = TfidfTransformer()
X_train = tfidfconverter.fit_transform(X_train).toarray()

In [41]:
X_test = vectorizer.transform(test_documents).toarray()
X_test = tfidfconverter.transform(X_test).toarray()

In [48]:
from sklearn.feature_extraction.text import TfidfVectorizer

# vectorizer = TfidfVectorizer (max_features=2500, min_df=7, max_df=0.8, stop_words=stopwords.words('english'))
# processed_features = vectorizer.fit_transform(processed_features).toarray()

vectorizer = TfidfVectorizer (max_features=1500, min_df=5, max_df=0.7, stop_words=stopwords.words('english'))
X_train = vectorizer.fit_transform(train_documents).toarray()
X_test = vectorizer.transform(test_documents).toarray()

In [ ]:
# from sklearn.model_selection import train_test_split

# X_train, X_test, y_train, y_test = train_test_split(processed_features, labels, test_size=0.2, random_state=0)


In [49]:
from sklearn.ensemble import RandomForestClassifier

text_classifier = RandomForestClassifier(n_estimators=200, random_state=0)
text_classifier.fit(X_train, y_train)


RandomForestClassifier(n_estimators=200, random_state=0)

In [50]:
predictions = text_classifier.predict(X_test)

In [51]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

print(confusion_matrix(y_test,predictions))
print(classification_report(y_test,predictions))
print(accuracy_score(y_test, predictions))

[[233  18   9  59]
 [  1 374  12   2]
 [  2  36 352   4]
 [ 36  24  10 181]]
              precision    recall  f1-score   support

           0       0.86      0.73      0.79       319
           1       0.83      0.96      0.89       389
           2       0.92      0.89      0.91       394
           3       0.74      0.72      0.73       251

    accuracy                           0.84      1353
   macro avg       0.83      0.83      0.83      1353
weighted avg       0.84      0.84      0.84      1353

0.8425720620842572


In [ ]:
categories = [
    "alt.atheism",
    "comp.graphics",
    "comp.sys.ibm.pc.hardware",
    "comp.sys.mac.hardware",
    "comp.windows.x",
    "misc.forsale",
    "rec.autos",
    "rec.motorcycles",
    "rec.sport.baseball",
    "rec.sport.hockey",
    "sci.crypt",
    "sci.electronics",
    "sci.med",
    "sci.space",
    "soc.religion.christian",
    "talk.politics.guns",
    "talk.politics.mideast",
    "talk.politics.misc",
    "talk.religion.misc",
]
newsgroups = datasets.fetch_20newsgroups(categories=categories)
y_true = newsgroups.target

## Try different values for tf idf, increase number of classes
## different classifiers
## change the vector representations


Vectorizing...
Coclustering...
Done in 12.09s. V-measure: 0.4431
MiniBatchKMeans...
Done in 5.15s. V-measure: 0.3177

Best biclusters:
----------------
bicluster 0 : 1961 documents, 4388 words
categories   : 23% talk.politics.guns, 18% talk.politics.misc, 17% sci.med
words        : gun, geb, guns, banks, gordon, clinton, pitt, cdt, surrender, veal

bicluster 1 : 1269 documents, 3558 words
categories   : 27% soc.religion.christian, 25% talk.politics.mideast, 24% alt.atheism
words        : god, jesus, christians, sin, objective, kent, belief, christ, faith, moral

bicluster 2 : 2201 documents, 2747 words
categories   : 18% comp.sys.mac.hardware, 17% comp.sys.ibm.pc.hardware, 16% comp.graphics
words        : voltage, board, dsp, packages, receiver, stereo, shipping, package, compression, image

bicluster 3 : 1773 documents, 2620 words
categories   : 27% rec.motorcycles, 23% rec.autos, 13% misc.forsale
words        : bike, car, dod, engine, motorcycle, ride, honda, bikes, helmet, bmw

bicl